Loading, selecting, calibrating, and exporting data
====================================


Setup
--------
When you've installed ixdat using `pip install ixdat` in your terminal or Anaconda prompt, no additional setup is needed!

Loading raw data
----------------------
This first block defines where the data is and tells us the contents of the folder.

In [ ]:
from pathlib import Path
data_directory = Path.home() / "Dropbox/ixdat_resources/tutorials_data/extended_platinum_ec"
[f.name for f in data_directory.iterdir()]

And now we're ready to read it with ixdat!

In [ ]:
from ixdat import Measurement

cv_meas = Measurement.read(
    data_directory / "01_demo_03_CVA_C01.mpt", reader="biologic"
)

cv_meas

These are the available readers (the list will be expanded!):

In [ ]:
from ixdat.readers import READER_CLASSES

READER_CLASSES

First, the actual data. The best way to get the numerical data (numpy arrays) is using `grab()`.

In [ ]:
cv_meas.grab("Ewe/V")  # "Ewe/V" is what Biologic calls working electrode potential in its files.

Grab gives two vectors. The first is time, the second is the variable.
You can plot them like this:
    

In [ ]:
from matplotlib import pyplot as plt  # the standard plotting library

t, v = cv_meas.grab("Ewe/V")
plt.plot(t, v)

plt.gca().set_xlim([13000, 15000])  # you can zoom in using an xaxis limit

Rather than using matplotlib to zoom in, we can also use ixdat to select the data.

In [ ]:
t, v = cv_meas.grab("Ewe/V", tspan=[13000, 15000])
plt.plot(t, v)

Beter yet, we can use ixdat to plot it for us.

In [ ]:
cv_meas.plot_measurement()
cv_meas.calibrate_RE(RE_vs_RHE=0.715)  # this calibrates the reference electrode potential!
cv_meas.plot_measurement(tspan=[13000, 14000])
cv_meas.plot_measurement(tspan=[16000, 17000])

**The other parts of the measurement**
-------------------------------------------------

In [ ]:
ocp_meas = Measurement.read(
    data_directory / "01_demo_02_OCV_C01.mpt", reader="biologic"
)
ocp_meas.plot_measurement()

cp_meas = Measurement.read(
    data_directory / "01_demo_04_CP_C01.mpt", reader="biologic"
)
cp_meas.plot_measurement()

Combining the measurements
-----------------------------------------

In [ ]:
combined_meas = cv_meas + ocp_meas + cp_meas
combined_meas.plot_measurement()

In [ ]:
combined_meas.plot(tspan=[16000, 17000])

If you need to separate them again
--------------------------------

In [ ]:
combined_meas.component_measurements

In [ ]:
part = combined_meas.component_measurements[2]
part

In [ ]:
part.plot_measurement(tspan=[14400, 17000])

Saving to ixdat
---------------------

In [ ]:
from ixdat.db import change_database

change_database("directory", project_name="21B19_demo")

In [ ]:
print(f"About to save {combined_meas}")
combined_meas.name = "extended_platinum_ec"

m_id = combined_meas.save()

print(f"saved to measurements table with id={m_id})")
print(f"... {combined_meas}, that is.")

In [ ]:
del(combined_meas)

loaded_meas = Measurement.get(m_id)

In [ ]:
loaded_meas.plot_measurement(tspan=[16000, 17000])

Selecting data and adjusting the starting time
-------------------------------------------------------------

In [ ]:
co_strip = loaded_meas.cut(tspan=[13300, 13800])

co_strip.plot_measurement()

In [ ]:
t_0 = co_strip.t[0]
print(f"the time starts at t={t_0}, which is annoying")

In [ ]:
co_strip.tstamp += 13300
t_0 = co_strip.t[0]
print(f"and now it starts at t={t_0}, which is nice!")

In [ ]:
from matplotlib import pyplot as plt

fig, [ax1, ax2] = plt.subplots(1, 2)
fig.set_figwidth(fig.get_figheight()*2.5)

co_strip.plot_measurement(
    axes=[ax2, ax2.twinx()], V_color="b", J_color="g"
)

co_strip.plot_vs_potential(ax=ax1)

In [ ]:
oxide_reduction = loaded_meas.cut(
    tspan=[16000, 17000], t_zero="start"
)

oxide_reduction.plot_measurement()

Exporting
-------------

In [ ]:
co_strip.export("./co_strip.csv")

In [ ]:
oxide_reduction.export("./oxide_reduction.csv")

In [ ]:
loaded_from_export = Measurement.read("./co_strip.csv", reader="ixdat")

In [ ]:
loaded_from_export.plot()